In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


here the objective is to fit the logit graph model to multiple different graph domains and then visualize the original vs fitted graphs

In [45]:

import sys
import os
sys.path.append('../')

#Graph imports
import src.graph as graph
import src.logit_estimator as estimator
import src.utils as utils
import src.model_selection as model_selection
import src.gic as gic
import src.param_estimator as pe
import src.graph as graph
import src.model_selection as ms

# usual imports
import matplotlib.pyplot as plt
import pickle
import math
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import gaussian_kde
import numpy as np
import pandas as pd
import seaborn as sns
import gc
import random
import networkx as nx
from numpy import errstate

from IPython.display import display
from pyvis.network import Network
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [46]:
FOLDER_OUTPUT = f'runs/fitted_graphs_comparison_twitter1'
out_files = os.listdir(FOLDER_OUTPUT)
out_files = sorted(out_files, key=lambda x: os.path.getctime(os.path.join(FOLDER_OUTPUT, x)), reverse=True)

out_files

['comparators_623623.pkl',
 'comparators_22751884.pkl',
 'comparators_54695584.pkl',
 'comparators_43003845.pkl',
 'comparators_19658936.pkl',
 'comparators_144211337.pkl',
 'comparators_110755733.pkl',
 'comparators_16606585.pkl',
 'comparators_65185224.pkl',
 'comparators_314038032.pkl',
 'comparators_18687625.pkl',
 'comparators_33080832.pkl',
 'comparators_15099384.pkl',
 'comparators_78391198.pkl',
 'comparators_54226675.pkl',
 'comparators_5773532.pkl',
 'comparators_28628055.pkl',
 'comparators_23503181.pkl',
 'comparators_134208594.pkl',
 'comparators_333881828.pkl',
 'comparators_33794839.pkl',
 'comparators_9855382.pkl',
 'comparators_258955239.pkl',
 'comparators_126067398.pkl',
 'comparators_549089429.pkl',
 'comparators_101903164.pkl',
 'comparators_58525541.pkl',
 'comparators_16674149.pkl',
 'comparators_16297193.pkl',
 'comparators_104324908.pkl',
 'comparators_17421011.pkl',
 'comparators_96545499.pkl',
 'comparators_262310943.pkl',
 'comparators_43858661.pkl',
 'compa

In [49]:
import pickle
#pick first id from out_files
id = out_files[0].split('_')[1]

with open(f'{FOLDER_OUTPUT}/comparators_{id}', 'rb') as f:
    comparators = pickle.load(f)


In [51]:
len(comparators)

19

In [52]:
dfs = []
for i in range(len(comparators)):
    dfs.append(comparators[i].summary_df)

In [53]:
dfs[1]

,graph_filename,model,gic_value,param,fit_success,nodes,edges,density,avg_clustering,avg_path_length,diameter,assortativity,num_components,largest_component_size
0,23503181.edges,Original,NaN,N/A,True,101,1455,0.288119,0.612132,1.786733,3,-0.173453,1,101
1,23503181.edges,LG,0.795880,"d=1, sigma=-5.4724",True,101,1506,0.298218,0.301566,1.701782,2,-0.027576,1,101
2,23503181.edges,BA,0.594682,5.0,True,101,480,0.095050,0.190592,2.227129,4,-0.097471,1,101
3,23503181.edges,ER,0.712607,0.2,True,101,994,0.196832,0.196234,1.819010,3,-0.020593,1,101
4,23503181.edges,WS,1.359712,20.2,True,101,1414,0.280000,0.267722,1.720000,2,-0.022361,1,101
5,23503181.edges,GRG,4.282978,1.0,True,101,4942,0.978614,0.981219,1.021386,2,-0.053043,1,101


In [54]:
pd.concat(dfs)

,graph_filename,model,gic_value,param,fit_success,nodes,edges,density,avg_clustering,avg_path_length,diameter,assortativity,num_components,largest_component_size
0,134208594.edges,Original,NaN,N/A,True,100,1677,0.338788,0.612483,1.692929,3,-0.101428,1,100
1,134208594.edges,LG,0.215938,"d=2, sigma=-51.9010",True,100,1672,0.337778,0.340221,1.662222,2,0.010013,1,100
2,134208594.edges,ER,0.408830,0.2,True,100,1008,0.203636,0.203689,1.807273,3,-0.088399,1,100
3,134208594.edges,BA,0.553002,5.0,True,100,475,0.095960,0.179941,2.224646,4,-0.114091,1,100
4,134208594.edges,WS,1.485660,20.178889,True,100,1700,0.343434,0.334949,1.656566,2,-0.028464,1,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,623623.edges,LG,0.131513,"d=2, sigma=-8.1152",True,106,395,0.070979,0.095330,2.525606,5,-0.020953,1,106
2,623623.edges,WS,0.737604,20.2,True,106,424,0.076190,0.052139,2.437376,4,0.012566,1,106
3,623623.edges,ER,1.400686,0.115556,True,106,642,0.115364,0.118718,2.106199,4,0.045675,1,106
4,623623.edges,BA,1.406585,5.0,True,106,505,0.090746,0.264261,2.198383,4,-0.182625,1,106


In [55]:
# Create a DataFrame with GIC values for each model and graph
gic_df = pd.DataFrame()

for df in dfs:
    graph_name = df['graph_filename'].iloc[0]
    # Get GIC values for each model (excluding Original which has NaN)
    model_gics = df[df['model'] != 'Original'].set_index('model')['gic_value']
    gic_df[graph_name] = model_gics

# Transpose so graphs are rows and models are columns
gic_df = gic_df.T
gic_df

model,LG,ER,BA,WS,GRG
134208594.edges,0.215938,0.408830,0.553002,1.485660,2.791326
23503181.edges,0.795880,0.712607,0.594682,1.359712,4.282978
28628055.edges,0.787652,0.636669,0.486718,1.248200,6.476412
5773532.edges,0.218005,0.599033,0.411241,1.117310,8.528437
54226675.edges,0.897812,0.883680,0.269791,1.302425,6.202168
78391198.edges,0.434204,0.391671,0.439353,0.959810,10.619165
15099384.edges,0.404820,0.416319,0.476408,0.953764,10.351676
33080832.edges,0.648200,0.731441,0.714442,0.325028,14.187453
18687625.edges,0.044660,0.804470,0.041369,0.686736,12.587218
314038032.edges,0.561572,0.387709,0.457059,1.246600,7.703593


In [56]:
# Create a dictionary to store rankings for each graph
rankings = {}

# Go through each df in dfs
for df in dfs:
    graph_name = df['graph_filename'].iloc[0]  # Get graph filename
    
    # Sort models by GIC value (excluding 'Original' which has NaN GIC)
    model_ranks = df[df['model'] != 'Original'].sort_values('gic_value')['model'].tolist()
    
    # Create rankings dictionary for this graph (1 is best since lowest GIC is best)
    rankings[graph_name] = {model: rank+1 for rank, model in enumerate(model_ranks)}

# Get unique models (excluding 'Original')
models = sorted(list(set([model for df in dfs for model in df['model'].unique() if model != 'Original'])))

# Create DataFrame with rankings
ranking_df = pd.DataFrame(rankings).T

# Ensure all models are present as columns
for model in models:
    if model not in ranking_df.columns:
        ranking_df[model] = np.nan

# Keep only the model columns in a consistent order
ranking_df = ranking_df[models]

print("Rankings by GIC value (lower is better):")
display(ranking_df)


Rankings by GIC value (lower is better):


,BA,ER,GRG,LG,WS
134208594.edges,3,2,5,1,4
23503181.edges,1,2,5,3,4
28628055.edges,1,2,5,3,4
5773532.edges,2,3,5,1,4
54226675.edges,1,2,5,3,4
78391198.edges,3,1,5,2,4
15099384.edges,3,2,5,1,4
33080832.edges,3,4,5,2,1
18687625.edges,1,4,5,2,3
314038032.edges,2,1,5,3,4


In [57]:
ranking_df.mean(axis=0)

BA     2.368421
ER     2.526316
GRG    5.000000
LG     1.789474
WS     3.315789
dtype: float64